# Saving and Loading Models

後のセッションでロードして実行できるように、モデルを保存することを望むかもしれない。  
最も簡単なそれを行う方法はBSON.jlによるものだ。

モデルを保存しよう。

In [1]:
using Flux

In [2]:
model = Chain(
    Dense(10,5,relu),
    Dense(5,2), softmax)

Chain(Dense(10, 5, NNlib.relu), Dense(5, 2), NNlib.softmax)

In [3]:
using BSON: @save

In [4]:
@save "mymodel.bson" model

再びロードしよう。

In [1]:
using Flux

In [2]:
using BSON:@load

In [6]:
@load "mymodel.bson" model

In [7]:
model

Chain(Dense(10, 5, NNlib.relu), Dense(5, 2), NNlib.softmax)

モデルは普通のJuliaの構造体なので、この目的のためには、どんなJuliaのストレージフォーマットを使っても良い。  
BSON.jlは特に良くサポートされていて、ほとんどの場合、前方互換性がある。（つまり、未来のバージョンのFluxで今保存したモデルがロードされる予定だということ）

> 保存したモデルの重みがGPUにあるなら、使えるGPUサポートがないとモデルは後でロードされない。  
> それを保存する前に`cpu(model)`でCPUにモデルを移動するのが最良。

## Saving Model Weights
---

場合によってはモデルパラメータ自体を保存して、君のコードの中でモデルの構造を再構築すると便利かもしれない。  
モデルパラメータを得るために`params(model)`が使える。  
追跡（tracking)を削除するために`data.(params)`を使うこともできる。

In [8]:
using Flux

In [9]:
model = Chain(
    Dense(10,5,relu),
    Dense(5,2), softmax)

Chain(Dense(10, 5, NNlib.relu), Dense(5, 2), NNlib.softmax)

In [16]:
weights = Tracker.data.(params(model))

4-element Array{Array{Float64,N} where N,1}:
 [-0.0728872 -0.451718 … 0.0186089 -0.507603; -0.605246 0.051562 … 0.471907 0.297695; … ; 0.167104 -0.0734559 … -0.0877545 -0.197722; -0.144643 0.291875 … -0.561696 -0.044596]
 [0.0, 0.0, 0.0, 0.0, 0.0]                                                                                                                                                     
 [-0.73627 0.45739 … -0.549217 -0.041836; 0.314933 0.617622 … 0.0132501 -0.627125]                                                                                             
 [0.0, 0.0]                                                                                                                                                                    

In [17]:
using BSON: @save

In [18]:
@save "mymodel.bson" weights

`Flux.loadparams!`でパラメータをロードしてモデルに戻すことが簡単にできる。

In [19]:
using Flux

In [20]:
model = Chain(
    Dense(10,5,relu),
    Dense(5,2), softmax)

Chain(Dense(10, 5, NNlib.relu), Dense(5, 2), NNlib.softmax)

In [21]:
using BSON: @load

In [22]:
@load "mymodel.bson" weights

In [23]:
Flux.loadparams!(model, weights)

作った新しい`model`は保存したパラメータと同一になる。

## Checkpointing
---

長いトレーニングの実行では、トレーニングが中断したときに再開できるように、定期的にモデルを保存するのを薦める。  
これは`train!`向けに適用されているcallbackの中でモデルを保存する事でできる。

In [25]:
using Flux: throttle
using BSON: @save

m = Chain(
    Dense(10,5,relu),
    Dense(5,2), softmax)

evalcb = throttle(30) do
    # lossを表示
    @show "model-checkpoint.bson" model
end

(::getfield(Flux, Symbol("#throttled#18")){getfield(Flux, Symbol("##throttled#10#14")){Bool,Bool,getfield(Main, Symbol("##3#4")),Int64}}) (generic function with 1 method)

これ（上のコード）は"`model-checkpoint.bson`"ファイルを30秒ごとに更新する。  

トレーニング中に一連のモデルを保存するのは、更に高度なものになる。

In [32]:
using Dates:now 

@save "model-$(now()).bson" model

上記は"`model-2018-03-06T02:57:10.41.bson`"ように一連のモデルを作り出す。  
今のテストセットのlossを保存もできる。だからオーバーフィットを始めたら、（例えば）モデルの古いコピーに戻すことも簡単だ。

In [100]:
# testloss stub
test_data, test_label = rand(10), [0 1]
testloss() = begin
    Flux.crossentropy(m(test_data), test_label)
end
@show testloss()

@save "model-$(now()).bson" model loss=testloss()

testloss() = 0.6931471805599453 (tracked)


UndefRefError: UndefRefError: access to undefined reference

モデルと共にオプティマイザの状態を保存して、止めた所から正確にトレーニングを再開することもできる。

[WIP]